In [2]:
import sklearn
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from scipy.stats import multivariate_normal # MVN not univariate

file_path = "data.csv"
df = pd.read_csv(file_path, index_col='year')

df.drop(['artists','id', 'name', 'release_date' ], axis = 1, inplace = True)

### Only look at decades from 50s to 10s (2020 not included)
l_drop = np.arange(1921,1950)
l_drop = np.append(l_drop,2020)
df.drop(labels=l_drop, axis=0, inplace = True)

enc = LabelEncoder()
labels = df.index
standardized_labels = np.array(labels)
enc.fit(df.index.unique())



lmao = df.index
y = enc.transform(standardized_labels)
Y = enc.transform(np.unique(standardized_labels))

y_decade = y//10
Y_decade = np.unique(y_decade)

enc.fit(df['explicit'].unique())
df['explicit'] = enc.transform(df['explicit'])

df.set_index(y_decade, inplace=True)
aa = df.index.value_counts().sort_index().to_numpy()
priors = aa/len(df.index)
check = np.sum(priors)
class_priors = np.diag(priors)
num_classes = len(Y_decade)
print(y_decade)
print(aa)
print(df.index.value_counts())

[0 0 0 ... 6 6 6]
[19950 20000 20000 20000 20000 20000 19900]
5    20000
4    20000
3    20000
2    20000
1    20000
0    19950
6    19900
dtype: int64


In [3]:
# for l in range(num_classes):
#     print(X[y_decade == l])


In [4]:
def regularized_cov(X, lambda_reg):
    n = X.shape[0]
    sigma = np.cov(X)
    # Selecting the regularization parameter should be performed using CV and a separate data subset
    # As I only went by training set performance (overfitting) in this problem, I settled on lambda=1/n
    sigma += lambda_reg * np.eye(n)
    return sigma

covariance = df.std()
mean = df.mean()
X = (df-df.mean())/df.std()

mu = df.groupby([df.index]).mean().to_numpy()
n = mu.shape[1]
Sigma = np.array([regularized_cov(X[y_decade == l].T,(1/n)) for l in range(num_classes)])
# Sigma = np.array([np.cov(X[y_decade == l].T) for l in range(num_classes)])
print(mu)
print(Sigma)

[[ 8.40149803e-01  4.77069233e-01  2.20807673e+05  2.87032280e-01
   8.62155388e-03  2.43156158e-01  5.03062657e+00  2.09645654e-01
  -1.47921127e+01  7.13233083e-01  9.94706767e+00  9.68375739e-02
   1.10601903e+02  4.76690994e-01]
 [ 6.21165633e-01  4.96651230e-01  2.10132680e+05  4.15795562e-01
   6.00000000e-04  1.54654999e-01  5.10565000e+00  2.09356285e-01
  -1.27372676e+01  7.54150000e-01  2.56291500e+01  5.88183550e-02
   1.15473647e+02  5.56917659e-01]
 [ 3.92205985e-01  5.26920000e-01  2.54870402e+05  5.38543794e-01
   3.20000000e-03  1.13695970e-01  5.11465000e+00  2.20958510e-01
  -1.13881736e+01  7.42850000e-01  3.50491000e+01  6.06646750e-02
   1.20126717e+02  5.87902195e-01]
 [ 2.86104236e-01  5.49658215e-01  2.52207315e+05  6.02474928e-01
   2.43500000e-02  1.18441519e-01  5.29995000e+00  2.05266970e-01
  -1.11292057e+01  7.11450000e-01  3.70011500e+01  6.29248650e-02
   1.21488896e+02  5.70081416e-01]
 [ 2.92137031e-01  5.71789510e-01  2.49228373e+05  5.94564894e-01
  

In [ ]:
C = len(priors)

class_cond_likelihoods = np.array([multivariate_normal.pdf(X, mu[j], Sigma[j]) for j in range(C)])

# Class Posterior
# P(yj | x) = p(x | yj) * P(yj) / p(x)
class_posteriors = class_priors.dot(class_cond_likelihoods)
print(np.max(class_cond_likelihoods))

decisions = np.argmax(class_posteriors, axis=0)

sample_class_counts = np.array([sum(y == j) for j in Y_decade])


conf_mat = np.zeros((C, C))
display_mat = np.zeros((C,C))
for i in range(C): # Each decision option
    for j in range(C): # Each class label
        ind_ij = np.argwhere((decisions==Y_decade[i]) & (y_decade==Y_decade[j]))
        display_mat[i, j] = len(ind_ij) # Average over class sample count
        conf_mat[i, j] = len(ind_ij)/sample_class_counts[j]

print("Confusion matrix:")
print(display_mat.astype(int))
print(np.sum(display_mat))

In [ ]:
print(Y)

In [ ]:
means = df.groupby(['year']).mean().to_numpy() 
print(means)

In [ ]:

# covariance = df.std()
# means = df.mean()



# X = (df-df.mean())/df.std()
# # print(X)
# indexer = df.index.values
# # print(df.head())



# columns = df.columns

# mu = []

# labels = set()
# for ax in df.index:
#     labels.add(ax)
# # print(labels)


    
# for index in indexer:
#     mu.append([np.mean(X[feature][index]) for feature in df])
# mu = np.array(mu)
# n = mu.shape[1]
# Sigma = []
# # print(mu)
# for index in indexer:
#     Sigma.append(np.cov([X[feature][index] for feature in df]))
# print(df.head())